# https://nlp100.github.io/ja/ch05.html

## 第5章: 係り受け解析  
日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

ステップ1: 依存関係のインストール  
sudo apt update  
sudo apt install -y build-essential curl file git  
sudo apt install -y mecab libmecab-dev mecab-ipadic-utf8  
sudo apt install -y zlib1g-dev libcurl4-openssl-dev libxml2-dev  

ステップ2: CRF++のインストール  
git clone https://github.com/taku910/crfpp.git  
cd crfpp  
./configure  
make  
sudo make install  
cd ..  

ステップ3: CaboChaのダウンロードとビルド  
git clone https://github.com/taku910/cabocha.git  
cd cabocha  
sudo apt-get install autoconf automake libtool  
autoreconf -i  
./configure --with-mecab-config=`which mecab-config` --with-charset=utf8  
make  
sudo make install  

swig_import_helper`について、以下の部分を修正
```
def swig_import_helper():
    import os  
    import importlib.util  
    spec = importlib.util.find_spec('_CaboCha', [os.path.dirname(__file__)])  
    if spec is None:  
        raise ImportError('_CaboCha module not found')  
    _mod = importlib.util.module_from_spec(spec)  
    spec.loader.exec_module(_mod)  
    return _mod  ```

In [2]:
import CaboCha
CBC = CaboCha.Parser()
divtext = []
with open("./datafiles/ai.ja.txt", "r") as f, open("./datafiles/ai.ja.txt.parsed", "w") as f2:
    lines = f.readlines()
    for text in lines:
        if "。" in text:
            temp = text.split("。")
            temp = [x + "。" for x in temp if x != '']
            divtext.extend(temp)
    for text in divtext:
        tree = CBC.parse(text)
        f2.write(tree.toString(CaboCha.FORMAT_LATTICE))

iconv_open is not supported


In [8]:
!head -5 ./datafiles/ai.ja.txt

人工知能

人工知能（じんこうちのう、、AI〈エーアイ〉）とは、「『計算（）』という概念と『コンピュータ（）』という道具を用いて『知能』を研究する計算機科学（）の一分野」を指す語。「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピューターに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。

『日本大百科全書(ニッポニカ)』の解説で、情報工学者・通信工学者の佐藤理史は次のように述べている。


In [6]:
!head -20 ./datafiles/ai.ja.txt.parsed

* 0 14D 1/1 -1.776924
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
* 1 14D 2/3 -1.776924
（	記号,括弧開,*,*,*,*,（,（,（
じん	名詞,一般,*,*,*,*,じん,ジン,ジン
こうち	名詞,一般,*,*,*,*,こうち,コウチ,コーチ
のう	助詞,終助詞,*,*,*,*,のう,ノウ,ノー
、	記号,読点,*,*,*,*,、,、,、
、	記号,読点,*,*,*,*,、,、,、
* 2 3D 0/0 0.592011
AI	名詞,一般,*,*,*,*,*
* 3 14D 1/5 -1.776924
〈	記号,括弧開,*,*,*,*,〈,〈,〈
エーアイ	名詞,固有名詞,一般,*,*,*,*
〉	記号,括弧閉,*,*,*,*,〉,〉,〉
）	記号,括弧閉,*,*,*,*,）,）,）
と	助詞,格助詞,引用,*,*,*,と,ト,ト
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
、	記号,読点,*,*,*,*,、,、,、


## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ

In [22]:
class Morph:
    def __init__(self, pos):
        """
        形態素を初期化します。
        :param pos: 形態素情報が格納されたリスト
        """
        self.surface = pos[0]  # 表層形
        self.base = pos[7]     # 基本形
        self.pos = pos[1]      # 品詞
        self.pos1 = pos[2]     # 品詞細分類1

    def __repr__(self):
        return f"Morph(surface='{self.surface}', base='{self.base}', pos='{self.pos}', pos1='{self.pos1}')"

    @classmethod # Morphクラスから直接メソッドを呼び出すために指定
    def parse_from_file(cls, file_path):
        """
        形態素解析結果を読み込み、文ごとの Morph オブジェクトのリストを生成します。
        :param file_path: ファイルのパス
        :return: 文ごとの Morph オブジェクトのリスト
        """
        morph_list = []
        sentence = []

        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()  # 空白や改行を除去
                if line == "EOS":  # 文の区切り
                    if sentence:
                        morph_list.append(sentence)
                        sentence = []
                elif not line.startswith("*"):  # 係り受け情報をスキップ
                    cols = line.split("\t")
                    if len(cols) > 1:
                        features = cols[1].split(",")
                        pos = [cols[0]] + features
                        sentence.append(cls(pos))  # クラスメソッドを使ってインスタンスを生成

        return morph_list


# ファイルパスを指定して実行
file_path = "./datafiles/ai.ja.txt.parsed"
morph_list = Morph.parse_from_file(file_path)

# 冒頭の文の形態素列を表示
if morph_list:
    print("冒頭の文の形態素列:")
    for morph in morph_list[0]:
        print(morph)

冒頭の文の形態素列:
Morph(surface='人工', base='人工', pos='名詞', pos1='一般')
Morph(surface='知能', base='知能', pos='名詞', pos1='一般')
Morph(surface='（', base='（', pos='記号', pos1='括弧開')
Morph(surface='じん', base='じん', pos='名詞', pos1='一般')
Morph(surface='こうち', base='こうち', pos='名詞', pos1='一般')
Morph(surface='のう', base='のう', pos='助詞', pos1='終助詞')
Morph(surface='、', base='、', pos='記号', pos1='読点')
Morph(surface='、', base='、', pos='記号', pos1='読点')
Morph(surface='AI', base='*', pos='名詞', pos1='一般')
Morph(surface='〈', base='〈', pos='記号', pos1='括弧開')
Morph(surface='エーアイ', base='*', pos='名詞', pos1='固有名詞')
Morph(surface='〉', base='〉', pos='記号', pos1='括弧閉')
Morph(surface='）', base='）', pos='記号', pos1='括弧閉')
Morph(surface='と', base='と', pos='助詞', pos1='格助詞')
Morph(surface='は', base='は', pos='助詞', pos1='係助詞')
Morph(surface='、', base='、', pos='記号', pos1='読点')
Morph(surface='「', base='「', pos='記号', pos1='括弧開')
Morph(surface='『', base='『', pos='記号', pos1='括弧開')
Morph(surface='計算', base='計算', pos='名詞', pos1='サ変接続')
Morph(su

## 41. 係り受け解析結果の読み込み（文節・係り受け）  
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ

In [25]:
class Chunk:
    def __init__(self, morphs, dst):
        """
        文節を表すクラス
        :param morphs: Morphオブジェクトのリスト
        :param dst: 係り先文節インデックス番号
        """
        self.morphs = morphs  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst        # 係り先文節インデックス番号
        self.srcs = []        # 係り元文節インデックス番号のリスト

    def __repr__(self):
        morph_text = ''.join([morph.surface for morph in self.morphs if morph.pos != "記号"])
        return f"Chunk(text='{morph_text}', dst={self.dst}, srcs={self.srcs})"


def parse_chunks(file_path):
    """
    係り受け解析結果を読み込み、文ごとの Chunk オブジェクトのリストを生成する。
    :param file_path: ファイルのパス
    :return: 文ごとの Chunk オブジェクトのリスト
    """
    sentences = []
    chunks = []
    morphs = []
    dst = -1

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == "EOS":
                if chunks:
                    for i, chunk in enumerate(chunks):
                        # dstが有効な範囲内であることを確認
                        if 0 <= chunk.dst < len(chunks):
                            chunks[chunk.dst].srcs.append(i)
                    sentences.append(chunks)
                chunks = []
            elif line.startswith("*"):
                if morphs:
                    chunks.append(Chunk(morphs, dst))
                    morphs = []
                _, idx, dst, *_ = line.split()
                dst = int(dst.rstrip("D"))
            else:
                cols = line.split("\t")
                if len(cols) > 1:
                    features = cols[1].split(",")
                    pos = [cols[0]] + features
                    morphs.append(Morph(pos))
        if morphs:
            chunks.append(Chunk(morphs, dst))

    return sentences


# ファイルパスを指定して実行
file_path = "./datafiles/ai.ja.txt.parsed"
chunk_list = parse_chunks(file_path)

# 冒頭の文の文節と係り先を表示
if chunk_list:
    print("冒頭の文の文節と係り先:")
    for i, chunk in enumerate(chunk_list[0]):
        # 文節の文字列を取得
        chunk_text = ''.join([m.surface for m in chunk.morphs if m.pos != "記号"])
        
        # 係り先の文字列を取得（範囲チェック付き）
        if 0 <= chunk.dst < len(chunk_list[0]):
            dst_text = ''.join([m.surface for m in chunk_list[0][chunk.dst].morphs if m.pos != "記号"])
        else:
            dst_text = "なし"
        
        print(f"文節 {i}: {chunk_text} -> {dst_text}")

冒頭の文の文節と係り先:
文節 0: 人工知能 -> なし
文節 1: じんこうちのう -> なし
文節 2: AI -> エーアイとは
文節 3: エーアイとは -> なし
文節 4: 計算という -> コンピュータという
文節 5: 概念と -> コンピュータという
文節 6: コンピュータという -> 道具を
文節 7: 道具を -> 用いて
文節 8: 用いて -> 研究する
文節 9: 知能を -> 研究する
文節 10: 研究する -> 計算機科学の
文節 11: 計算機科学の -> 一分野を
文節 12: 一分野を -> 指す
文節 13: 指す -> なし


## 42. 係り元と係り先の文節の表示  
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [33]:
class Chunk:
    def __init__(self, morphs, dst):
        """
        文節を表すクラス
        :param morphs: Morphオブジェクトのリスト
        :param dst: 係り先文節インデックス番号
        """
        self.morphs = morphs  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst        # 係り先文節インデックス番号
        self.srcs = []        # 係り元文節インデックス番号のリスト

    def __repr__(self):
        morph_text = ''.join([morph.surface for morph in self.morphs if morph.pos != "記号"])
        return f"Chunk(text='{morph_text}', dst={self.dst}, srcs={self.srcs})"


def parse_chunks(file_path):
    """
    係り受け解析結果を読み込み、文ごとの Chunk オブジェクトのリストを生成する。
    :param file_path: ファイルのパス
    :return: 文ごとの Chunk オブジェクトのリスト
    """
    sentences = []
    chunks = []
    morphs = []
    dst = -1

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == "EOS":
                if chunks:
                    for i, chunk in enumerate(chunks):
                        # dstが有効な範囲内であることを確認
                        if 0 <= chunk.dst < len(chunks):
                            chunks[chunk.dst].srcs.append(i)
                    sentences.append(chunks)
                chunks = []
            elif line.startswith("*"):
                if morphs:
                    chunks.append(Chunk(morphs, dst))
                    morphs = []
                _, idx, dst, *_ = line.split()
                dst = int(dst.rstrip("D"))
            else:
                cols = line.split("\t")
                if len(cols) > 1:
                    features = cols[1].split(",")
                    pos = [cols[0]] + features
                    morphs.append(Morph(pos))
        if morphs:
            chunks.append(Chunk(morphs, dst))

    return sentences


# ファイルパスを指定して実行
file_path = "./datafiles/ai.ja.txt.parsed"
chunk_list = parse_chunks(file_path)

# 冒頭の文の文節と係り先を表示
if chunk_list:
    print("冒頭3文の文節と係り先:")
    for sentence_idx, sentence in enumerate(chunk_list[:3]):  # 冒頭3文
        print(f"\n【文 {sentence_idx + 1}】")
        for i, chunk in enumerate(sentence):
            # 文節の文字列を取得
            chunk_text = ''.join([m.surface for m in chunk.morphs if m.pos != "記号"])
            
            # 係り先の文字列を取得（範囲チェック付き）
            if 0 <= chunk.dst < len(sentence):
                dst_text = ''.join([m.surface for m in sentence[chunk.dst].morphs if m.pos != "記号"])
            else:
                dst_text = "なし"
            
            print(f"文節 {i}: {chunk_text}\t{dst_text}")

冒頭3文の文節と係り先:

【文 1】
文節 0: 人工知能	なし
文節 1: じんこうちのう	なし
文節 2: AI	エーアイとは
文節 3: エーアイとは	なし
文節 4: 計算という	コンピュータという
文節 5: 概念と	コンピュータという
文節 6: コンピュータという	道具を
文節 7: 道具を	用いて
文節 8: 用いて	研究する
文節 9: 知能を	研究する
文節 10: 研究する	計算機科学の
文節 11: 計算機科学の	一分野を
文節 12: 一分野を	指す
文節 13: 指す	なし

【文 2】
文節 0: 語	なし
文節 1: 言語の	理解や
文節 2: 理解や	理解や
文節 3: 推論	推論
文節 4: 問題解決などの	問題解決などの
文節 5: 知的行動を	人間に
文節 6: 人間に	人間に
文節 7: 代わって	コンピューターに
文節 8: コンピューターに	コンピューターに
文節 9: 行わせる	行わせる
文節 10: 技術または	実現に関する
文節 11: 計算機コンピュータによる	知的な
文節 12: 知的な	知的な
文節 13: 情報処理システムの	設計や
文節 14: 設計や	設計や
文節 15: 実現に関する	実現に関する
文節 16: 研究分野とも	研究分野とも

【文 3】
文節 0: される	なし


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出  
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [2]:
# Morphクラスの定義
class Morph:
    def __init__(self, surface, base, pos, pos1):
        """
        形態素を表すクラス
        :param surface: 表層形
        :param base: 基本形
        :param pos: 品詞
        :param pos1: 品詞細分類1
        """
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

    def __repr__(self):
        return f"Morph(surface='{self.surface}', base='{self.base}', pos='{self.pos}', pos1='{self.pos1}')"


# Chunkクラスの定義
class Chunk:
    def __init__(self, morphs, dst):
        """
        文節を表すクラス
        :param morphs: Morphオブジェクトのリスト
        :param dst: 係り先文節インデックス番号
        """
        self.morphs = morphs  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst        # 係り先文節インデックス番号
        self.srcs = []        # 係り元文節インデックス番号のリスト

    def __repr__(self):
        morph_text = ''.join([morph.surface for morph in self.morphs if morph.pos != "記号"])
        return f"Chunk(text='{morph_text}', dst={self.dst}, srcs={self.srcs})"


def parse_chunks(file_path):
    """
    係り受け解析結果を読み込み、文ごとの Chunk オブジェクトのリストを生成する。
    :param file_path: ファイルのパス
    :return: 文ごとの Chunk オブジェクトのリスト
    """
    sentences = []
    chunks = []
    morphs = []
    dst = -1

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == "EOS":
                if chunks:
                    for i, chunk in enumerate(chunks):
                        # dstが有効な範囲内であることを確認
                        if 0 <= chunk.dst < len(chunks):
                            chunks[chunk.dst].srcs.append(i)
                    sentences.append(chunks)
                chunks = []
                morphs = []
                dst = -1
            elif line.startswith("*"):
                if morphs:
                    chunks.append(Chunk(morphs, dst))
                    morphs = []
                parts = line.split()
                dst = int(parts[2].rstrip("D"))
            else:
                cols = line.split("\t")
                if len(cols) > 1:
                    surface = cols[0]
                    feature = cols[1].split(",")
                    base = feature[6]
                    pos = feature[0]
                    pos1 = feature[1]
                    morphs.append(Morph(surface, base, pos, pos1))
        # 最後の文の処理
        if morphs:
            chunks.append(Chunk(morphs, dst))
        if chunks:
            for i, chunk in enumerate(chunks):
                if 0 <= chunk.dst < len(chunks):
                    chunks[chunk.dst].srcs.append(i)
            sentences.append(chunks)

    return sentences


# ファイルパスを指定して解析を実行
file_path = "./datafiles/ai.ja.txt.parsed"
chunk_list = parse_chunks(file_path)

# 名詞を含む文節が動詞を含む文節に係る場合を抽出してタブ区切りで出力
with open("./datafiles/noun_to_verb_dependencies.txt", "w", encoding="utf-8") as output_file:
    for sentence in chunk_list:
        for chunk in sentence:
            # 文節に名詞が含まれているかチェック
            if any(morph.pos == "名詞" for morph in chunk.morphs):
                # 係り先が存在するか確認
                if 0 <= chunk.dst < len(sentence):
                    dst_chunk = sentence[chunk.dst]
                    # 係り先の文節に動詞が含まれているかチェック
                    if any(morph.pos == "動詞" for morph in dst_chunk.morphs):
                        # 記号を除いた文節の文字列を取得
                        src_text = ''.join([morph.surface for morph in chunk.morphs if morph.pos != "記号"])
                        dst_text = ''.join([morph.surface for morph in dst_chunk.morphs if morph.pos != "記号"])
                        # タブ区切りで出力
                        output_file.write(f"{src_text}\t{dst_text}\n")

## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

* 日本語の文字が文字化けしている。解消方法不明

In [12]:
import graphviz

# Morphクラスの定義が必要です。以下は仮の定義です。
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __repr__(self):
        return f"Morph(surface='{self.surface}', base='{self.base}', pos='{self.pos}', pos1='{self.pos1}')"

class Chunk:
    def __init__(self, morphs, dst):
        """
        文節を表すクラス
        :param morphs: Morphオブジェクトのリスト
        :param dst: 係り先文節インデックス番号
        """
        self.morphs = morphs  # 形態素（Morphオブジェクト）のリスト
        self.dst = dst        # 係り先文節インデックス番号
        self.srcs = []        # 係り元文節インデックス番号のリスト

    def __repr__(self):
        morph_text = ''.join([morph.surface for morph in self.morphs if morph.pos != "記号"])
        return f"Chunk(text='{morph_text}', dst={self.dst}, srcs={self.srcs})"


def parse_chunks(file_path):
    """
    係り受け解析結果を読み込み、文ごとの Chunk オブジェクトのリストを生成する。
    :param file_path: ファイルのパス
    :return: 文ごとの Chunk オブジェクトのリスト
    """
    sentences = []
    chunks = []
    morphs = []
    dst = -1

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line == "EOS":
                if chunks:
                    for i, chunk in enumerate(chunks):
                        # dstが有効な範囲内であることを確認
                        if 0 <= chunk.dst < len(chunks):
                            chunks[chunk.dst].srcs.append(i)
                    sentences.append(chunks)
                chunks = []
            elif line.startswith("*"):
                if morphs:
                    chunks.append(Chunk(morphs, dst))
                    morphs = []
                parts = line.split()
                if len(parts) >= 3:
                    _, idx, dst_str = parts[:3]
                    dst = int(dst_str.rstrip("D"))
                else:
                    dst = -1
            else:
                cols = line.split("\t")
                if len(cols) > 1:
                    surface = cols[0]
                    features = cols[1].split(",")
                    if len(features) >= 7:
                        base = features[6]
                    else:
                        base = "*"
                    pos = features[0]
                    pos1 = features[1]
                    morphs.append(Morph(surface, base, pos, pos1))
        if morphs:
            chunks.append(Chunk(morphs, dst))

    return sentences

def visualize_dependency_tree(sentence, file_name="dependency_tree"):
    """
    1文の係り受け木をGraphvizで可視化する。
    :param sentence: 文（Chunkオブジェクトのリスト）
    :param file_name: 出力ファイル名
    """
    dot = graphviz.Digraph(format="png")
    # DPIの設定
    dot.attr(dpi="300")
    # ノードのフォントを設定
    dot.attr('node', fontname="Noto Sans CJK JP")  

    for i, chunk in enumerate(sentence):
        # ノードのラベルは文節のテキスト
        chunk_text = ''.join([m.surface for m in chunk.morphs if m.pos != "記号"])
        dot.node(str(i), chunk_text)

        # 係り先がある場合、エッジを追加
        if 0 <= chunk.dst < len(sentence):
            dot.edge(str(i), str(chunk.dst))

    # グラフの出力と表示
    output_path = dot.render(file_name)
    print(f"グラフが以下に保存されました: {output_path}")

# 使用例
# 最初の文の係り受け木を可視化
file_path = "./datafiles/ai.ja.txt.parsed"
chunk_lists = parse_chunks(file_path)
if chunk_lists:
    first_sentence = chunk_lists[0]
    visualize_dependency_tree(first_sentence)

グラフが以下に保存されました: dependency_tree.png


## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

動詞を含む文節において，最左の動詞の基本形を述語とする
述語に係る助詞を格とする
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

In [17]:
import re
from collections import defaultdict

class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs      # Morphのリスト
        self.dst = dst            # 係り先のチャンク番号
        self.srcs = []            # 係り元のチャンク番号

def parse_cabocha(filename):
    sentences = []
    with open(filename, 'r', encoding='utf-8') as f:
        chunks = {}
        morphs = []
        dst = -1
        idx = -1  # 初期化
        for line in f:
            line = line.strip()
            if line.startswith('*'):
                if morphs:
                    chunks[idx] = Chunk(morphs, dst)
                    morphs = []
                parts = line.split()
                idx = int(parts[1])
                # 修正: '-' が付く場合と付かない場合の両方に対応
                match = re.search(r'(-?\d+)D', parts[2])
                if match:
                    dst = int(match.group(1))
                else:
                    # マッチしない場合のエラーハンドリング
                    print(f"Warning: 予期せぬ形式の依存先: {parts[2]}")
                    dst = -1
                chunks[idx] = Chunk([], dst)
            elif line == 'EOS':
                if chunks:
                    # 係り元を集める
                    for idx_key, chunk in chunks.items():
                        if chunk.dst != -1:
                            if chunk.dst in chunks:
                                chunks[chunk.dst].srcs.append(idx_key)
                            else:
                                print(f"Warning: 存在しないチャンク番号 {chunk.dst} への依存")
                    sentences.append(chunks)
                    chunks = {}
            else:
                if '\t' in line:
                    surface, feature = line.split('\t')
                    features = feature.split(',')
                    if len(features) >= 7:
                        morph = Morph(surface, features[6], features[0], features[1])
                        chunks[idx].morphs.append(morph)
    return sentences

def extract_predicate_case(sentences):
    results = []
    for chunks in sentences:
        predicate = None
        cases = []
        predicate_chunk = None
        # 述語を探す（最左の動詞）
        for idx in sorted(chunks.keys()):
            chunk = chunks[idx]
            for morph in chunk.morphs:
                if morph.pos == '動詞':
                    predicate = morph.base
                    predicate_chunk = chunk
                    break
            if predicate:
                break
        if predicate and predicate_chunk:
            # 述語に係る助詞を探す
            for src in predicate_chunk.srcs:
                src_chunk = chunks[src]
                for morph in src_chunk.morphs:
                    if morph.pos == '助詞':
                        cases.append(morph.surface)
                        break  # 一つの助詞で十分
            if cases:
                # 辞書順に並べる
                cases = sorted(cases)
                # 重複を除去
                cases = list(dict.fromkeys(cases))
                results.append(f"{predicate}\t{' '.join(cases)}")
            else:
                # 助詞がない場合の処理
                results.append(f"{predicate}\t")
    return results

# 入力ファイルと出力ファイルを指定
input_filename = './datafiles/ai.ja.txt.parsed'  # 実際のパスに変更してください
output_filename = './datafiles/predicate_cases.txt'  # 出力ファイル名

# コーパスを解析
sentences = parse_cabocha(input_filename)
results = extract_predicate_case(sentences)

# 出力ファイルに書き込む
with open(output_filename, 'w', encoding='utf-8') as out_f:
    for line in results:
        out_f.write(line + '\n')

print(f"出力を {output_filename} に保存しました。")


出力を ./datafiles/predicate_cases.txt に保存しました。


In [20]:
!head -n 10 ./datafiles/predicate_cases.txt

用いる	を
代わる	に を
述べる	で の は
する	で を
する	を
する	で に により
用いる	を
呼ぶ	も
する	を
知る	として も


In [19]:
!sort ./datafiles/predicate_cases.txt | uniq -c | sort -nr > ./datafiles/predicate_case_frequencies.txt

In [21]:
!head -n 10 ./datafiles/predicate_case_frequencies.txt

     15 する	を
      8 よる	に
      7 する	が
      5 する	に を
      5 する	に
      5 する	が に
      4 する	は を
      3 基づく	に
      3 向ける	に
      3 受ける	を


## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

In [22]:
import re
from collections import defaultdict

class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs      # Morphのリスト
        self.dst = dst            # 係り先のチャンク番号
        self.srcs = []            # 係り元のチャンク番号

    def surface(self):
        """文節内の表層形を連結して返す"""
        return ''.join(m.surface for m in self.morphs if m.pos != '記号')

def parse_cabocha(filename):
    sentences = []
    with open(filename, 'r', encoding='utf-8') as f:
        chunks = {}
        morphs = []
        dst = -1
        idx = -1  # 初期化
        for line in f:
            line = line.strip()
            if line.startswith('*'):
                if morphs:
                    chunks[idx] = Chunk(morphs, dst)
                    morphs = []
                parts = line.split()
                idx = int(parts[1])
                match = re.search(r'(-?\d+)D', parts[2])
                if match:
                    dst = int(match.group(1))
                else:
                    print(f"Warning: 予期せぬ形式の依存先: {parts[2]}")
                    dst = -1
                chunks[idx] = Chunk([], dst)
            elif line == 'EOS':
                if chunks:
                    for idx_key, chunk in chunks.items():
                        if chunk.dst != -1:
                            if chunk.dst in chunks:
                                chunks[chunk.dst].srcs.append(idx_key)
                            else:
                                print(f"Warning: 存在しないチャンク番号 {chunk.dst} への依存")
                    sentences.append(chunks)
                    chunks = {}
            else:
                if '\t' in line:
                    surface, feature = line.split('\t')
                    features = feature.split(',')
                    if len(features) >= 7:
                        morph = Morph(surface, features[6], features[0], features[1])
                        chunks[idx].morphs.append(morph)
    return sentences

def extract_predicate_case_with_chunks(sentences):
    results = []
    for chunks in sentences:
        for idx in sorted(chunks.keys()):
            chunk = chunks[idx]
            predicate = None
            cases = []
            phrases = []
            for morph in chunk.morphs:
                if morph.pos == '動詞':
                    predicate = morph.base
                    break
            if predicate:
                for src in chunk.srcs:
                    src_chunk = chunks[src]
                    for morph in src_chunk.morphs:
                        if morph.pos == '助詞':
                            cases.append(morph.surface)
                            phrases.append(src_chunk.surface())
                            break
                if cases:
                    cases, phrases = zip(*sorted(zip(cases, phrases)))  # 助詞でソート
                    results.append(f"{predicate}\t{' '.join(cases)}\t{' '.join(phrases)}")
    return results

# 入力ファイルと出力ファイルを指定
input_filename = './datafiles/ai.ja.txt.parsed'  # 実際のパスに変更してください
output_filename = './datafiles/predicate_cases_with_chunks.txt'  # 出力ファイル名

# コーパスを解析
sentences = parse_cabocha(input_filename)
results = extract_predicate_case_with_chunks(sentences)

# 出力ファイルに書き込む
with open(output_filename, 'w', encoding='utf-8') as out_f:
    for line in results:
        out_f.write(line + '\n')

print(f"出力を {output_filename} に保存しました。")

出力を ./datafiles/predicate_cases_with_chunks.txt に保存しました。


## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．

In [23]:
import re
from collections import defaultdict

class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface  # 表層形
        self.base = base        # 基本形
        self.pos = pos          # 品詞
        self.pos1 = pos1        # 品詞細分類1

class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs      # Morphのリスト
        self.dst = dst            # 係り先のチャンク番号
        self.srcs = []            # 係り元のチャンク番号

    def surface(self):
        """文節内の表層形を連結して返す"""
        return ''.join(m.surface for m in self.morphs if m.pos != '記号')

    def has_sahen_wo(self):
        """サ変接続名詞+を（助詞）を含むか確認"""
        for i in range(len(self.morphs) - 1):
            if (self.morphs[i].pos1 == 'サ変接続' and
                self.morphs[i + 1].surface == 'を' and
                self.morphs[i + 1].pos == '助詞'):
                return True
        return False

    def sahen_wo_surface(self):
        """サ変接続名詞+をの表層形を返す"""
        for i in range(len(self.morphs) - 1):
            if (self.morphs[i].pos1 == 'サ変接続' and
                self.morphs[i + 1].surface == 'を' and
                self.morphs[i + 1].pos == '助詞'):
                return self.morphs[i].surface + self.morphs[i + 1].surface
        return ''

def parse_cabocha(filename):
    sentences = []
    with open(filename, 'r', encoding='utf-8') as f:
        chunks = {}
        morphs = []
        dst = -1
        idx = -1  # 初期化
        for line in f:
            line = line.strip()
            if line.startswith('*'):
                if morphs:
                    chunks[idx] = Chunk(morphs, dst)
                    morphs = []
                parts = line.split()
                idx = int(parts[1])
                match = re.search(r'(-?\d+)D', parts[2])
                if match:
                    dst = int(match.group(1))
                else:
                    print(f"Warning: 予期せぬ形式の依存先: {parts[2]}")
                    dst = -1
                chunks[idx] = Chunk([], dst)
            elif line == 'EOS':
                if chunks:
                    for idx_key, chunk in chunks.items():
                        if chunk.dst != -1:
                            if chunk.dst in chunks:
                                chunks[chunk.dst].srcs.append(idx_key)
                            else:
                                print(f"Warning: 存在しないチャンク番号 {chunk.dst} への依存")
                    sentences.append(chunks)
                    chunks = {}
            else:
                if '\t' in line:
                    surface, feature = line.split('\t')
                    features = feature.split(',')
                    if len(features) >= 7:
                        morph = Morph(surface, features[6], features[0], features[1])
                        chunks[idx].morphs.append(morph)
    return sentences

def extract_sahen_case(sentences):
    results = []
    for chunks in sentences:
        for idx in sorted(chunks.keys()):
            chunk = chunks[idx]
            if not chunk.has_sahen_wo():
                continue

            sahen_wo = chunk.sahen_wo_surface()
            if not sahen_wo:
                continue

            dst = chunk.dst
            if dst == -1 or chunks[dst] is None:
                continue

            predicate_chunk = chunks[dst]
            predicate = None
            for morph in predicate_chunk.morphs:
                if morph.pos == '動詞':
                    predicate = morph.base
                    break

            if not predicate:
                continue

            full_predicate = sahen_wo + predicate

            cases = []
            phrases = []
            for src in predicate_chunk.srcs:
                src_chunk = chunks[src]
                for morph in src_chunk.morphs:
                    if morph.pos == '助詞':
                        cases.append(morph.surface)
                        phrases.append(src_chunk.surface())
                        break

            if cases:
                cases, phrases = zip(*sorted(zip(cases, phrases)))  # 助詞でソート
                results.append(f"{full_predicate}\t{' '.join(cases)}\t{' '.join(phrases)}")
    return results

# 入力ファイルと出力ファイルを指定
input_filename = './datafiles/ai.ja.txt.parsed'  # 実際のパスに変更してください
output_filename = './datafiles/sahen_cases.txt'  # 出力ファイル名

# コーパスを解析
sentences = parse_cabocha(input_filename)
results = extract_sahen_case(sentences)

# 出力ファイルに書き込む
with open(output_filename, 'w', encoding='utf-8') as out_f:
    for line in results:
        out_f.write(line + '\n')

print(f"出力を {output_filename} に保存しました。")

出力を ./datafiles/sahen_cases.txt に保存しました。


## 48. 名詞から根へのパスの抽出
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

- 各文節は（表層形の）形態素列で表現する
- パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．

In [2]:
import re

def parse_cabocha(file_path):
    """
    CaboCha形式の解析結果を逐行処理して文節ごとの情報を生成。
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        chunks = []
        chunk = {"morphs": [], "dst": -1, "srcs": []}
        for line in f:
            if line.startswith('*'):
                if chunk["morphs"]:
                    chunks.append(chunk)
                    chunk = {"morphs": [], "dst": -1, "srcs": []}
                dst = int(re.search(r'(\d+)D', line).group(1))
                chunk["dst"] = dst
            elif line.strip() == 'EOS':
                if chunk["morphs"]:
                    chunks.append(chunk)
                for i, ch in enumerate(chunks):
                    if ch["dst"] != -1:
                        chunks[ch["dst"]]["srcs"].append(i)
                yield chunks
                chunks = []
            else:
                surface, features = line.split('\t')
                features = features.split(',')
                chunk["morphs"].append({"surface": surface, "pos": features[0]})

def extract_paths(chunks, output_file):
    """
    名詞を含む文節から構文木の根に至るパスを抽出してファイルに書き出す。
    """
    with open(output_file, 'a', encoding='utf-8') as f:
        for i, chunk in enumerate(chunks):
            if any(m["pos"] == "名詞" for m in chunk["morphs"]):  # 名詞を含む文節か確認
                path = []
                visited = set()  # 無限ループ防止用セット
                current = i
                while current != -1:
                    if current in visited:
                        break
                    visited.add(current)
                    surface = "".join(m["surface"] for m in chunks[current]["morphs"])
                    path.append(surface)
                    current = chunks[current]["dst"]
                f.write(" -> ".join(path) + '\n')

# 実行
file_path = './datafiles/ai.ja.txt.parsed'
output_file = './datafiles/output_paths.txt'

# 出力ファイルを初期化
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("")

try:
    for chunks in parse_cabocha(file_path):
        extract_paths(chunks, output_file)
    print(f"処理が完了しました。結果は {output_file} に出力されました。")
except FileNotFoundError:
    print(f"ファイルが見つかりません: {file_path}")
except Exception as e:
    print(f"エラーが発生しました: {e}")

処理が完了しました。結果は ./datafiles/output_paths.txt に出力されました。


## 49. 名詞間の係り受けパスの抽出
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がi
とj
（i<j
）のとき，係り受けパスは以下の仕様を満たすものとする．

問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
文節i
とj
に含まれる名詞句はそれぞれ，XとYに置換する
また，係り受けパスの形状は，以下の2通りが考えられる．

文節i
から構文木の根に至る経路上に文節j
が存在する場合: 文節i
から文節j
のパスを表示
上記以外で，文節i
と文節j
から構文木の根に至る経路上で共通の文節k
で交わる場合: 文節i
から文節k
に至る直前のパスと文節j
から文節k
に至る直前までのパス，文節k
の内容を” | “で連結して表示
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．

In [ ]:
import re

def parse_cabocha(file_path):
    """
    CaboCha形式の解析結果を逐行処理して文節ごとの情報を生成。
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        chunks = []
        chunk = {"morphs": [], "dst": -1, "srcs": []}
        for line in f:
            if line.startswith('*'):
                if chunk["morphs"]:
                    chunks.append(chunk)
                    chunk = {"morphs": [], "dst": -1, "srcs": []}
                dst = int(re.search(r'(\d+)D', line).group(1))
                chunk["dst"] = dst
            elif line.strip() == 'EOS':
                if chunk["morphs"]:
                    chunks.append(chunk)
                for i, ch in enumerate(chunks):
                    if ch["dst"] != -1:
                        chunks[ch["dst"]]["srcs"].append(i)
                yield chunks
                chunks = []
            else:
                surface, features = line.split('\t')
                features = features.split(',')
                chunk["morphs"].append({"surface": surface, "pos": features[0]})

def replace_noun(chunk, replacement):
    """
    文節内の最初の名詞を指定された文字列に置換。
    """
    replaced = False
    result = []
    for morph in chunk["morphs"]:
        if morph["pos"] == "名詞" and not replaced:
            result.append(replacement)
            replaced = True
        else:
            result.append(morph["surface"])
    return "".join(result)

def find_path(chunks, start):
    """
    文節 start から根に至るパスを取得。
    """
    path = []
    current = start
    while current != -1:
        path.append(current)
        current = chunks[current]["dst"]
    return path

def write_paths_to_file(chunks, file_obj):
    """
    文中の名詞句ペアを結ぶ最短係り受けパスを計算して即時ファイルに出力。
    """
    for i in range(len(chunks)):
        for j in range(i + 1, len(chunks)):
            if any(m["pos"] == "名詞" for m in chunks[i]["morphs"]) and \
               any(m["pos"] == "名詞" for m in chunks[j]["morphs"]):

                path_i = find_path(chunks, i)
                path_j = find_path(chunks, j)

                # 文節 i から j に直接つながる場合
                if j in path_i:
                    path = path_i[:path_i.index(j) + 1]
                    path_str = " -> ".join(
                        replace_noun(chunks[k], "X" if k == i else "Y" if k == j else "".join(m["surface"] for m in chunks[k]["morphs"]))
                        for k in path
                    )
                    file_obj.write(path_str + '\n')
                else:
                    # 文節 i と j の経路が交差する場合
                    common = set(path_i) & set(path_j)
                    if common:
                        common_node = min(common, key=path_i.index)
                        path_i_to_common = path_i[:path_i.index(common_node)]
                        path_j_to_common = path_j[:path_j.index(common_node)]

                        path_i_str = " -> ".join(
                            replace_noun(chunks[k], "X" if k == i else "".join(m["surface"] for m in chunks[k]["morphs"]))
                            for k in path_i_to_common
                        )
                        path_j_str = " -> ".join(
                            replace_noun(chunks[k], "Y" if k == j else "".join(m["surface"] for m in chunks[k]["morphs"]))
                            for k in path_j_to_common
                        )
                        common_str = "".join(m["surface"] for m in chunks[common_node]["morphs"])
                        file_obj.write(f"{path_i_str} | {path_j_str} | {common_str}\n")

# 実行
file_path = './datafiles/ai.ja.txt.parsed'
output_file = './datafiles/output_noun_paths.txt'

# 出力ファイルを初期化
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("")

try:
    with open(output_file, 'a', encoding='utf-8') as out_f:
        for chunks in parse_cabocha(file_path):
            write_paths_to_file(chunks, out_f)
    print(f"処理が完了しました。結果は {output_file} に出力されました。")
except FileNotFoundError:
    print(f"ファイルが見つかりません: {file_path}")
except Exception as e:
    print(f"エラーが発生しました: {e}")